In [2]:
import tensorflow as tf

# Training samples path, change to your local path
# # training_samples_file_path = tf.keras.utils.get_file("trainingSamples.csv",
#                                                      "file:///Users/zhewang/Workspace/SparrowRecSys/src/main"
#                                                      "/resources/webroot/sampledata/trainingSamples.csv")
# # Test samples path, change to your local path
# # test_samples_file_path = tf.keras.utils.get_file("testSamples.csv",
#                                                  "file:///Users/zhewang/Workspace/SparrowRecSys/src/main"
#                                                  "/resources/webroot/sampledata/testSamples.csv")


# load sample as tf dataset
def get_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=12,
        label_name='label',
        na_value="0",
        num_epochs=1,
        ignore_errors=True)
    return dataset


# split as test dataset and training dataset
train_dataset = get_dataset('./sampledata/trainingSamples.csv')
test_dataset = get_dataset('./sampledata/testSamples.csv')

# Config
RECENT_MOVIES = 5  # userRatedMovie{1-5},最近看过的5部 喜欢的电影
EMBEDDING_SIZE = 10

# define input for keras model
inputs = {
    'movieAvgRating': tf.keras.layers.Input(name='movieAvgRating', shape=(), dtype='float32'),
    'movieRatingStddev': tf.keras.layers.Input(name='movieRatingStddev', shape=(), dtype='float32'),
    'movieRatingCount': tf.keras.layers.Input(name='movieRatingCount', shape=(), dtype='int32'),
    'userAvgRating': tf.keras.layers.Input(name='userAvgRating', shape=(), dtype='float32'),
    'userRatingStddev': tf.keras.layers.Input(name='userRatingStddev', shape=(), dtype='float32'),
    'userRatingCount': tf.keras.layers.Input(name='userRatingCount', shape=(), dtype='int32'),
    'releaseYear': tf.keras.layers.Input(name='releaseYear', shape=(), dtype='int32'),

    'movieId': tf.keras.layers.Input(name='movieId', shape=(), dtype='int32'),
    'userId': tf.keras.layers.Input(name='userId', shape=(), dtype='int32'),
    'userRatedMovie1': tf.keras.layers.Input(name='userRatedMovie1', shape=(), dtype='int32'),
    'userRatedMovie2': tf.keras.layers.Input(name='userRatedMovie2', shape=(), dtype='int32'),
    'userRatedMovie3': tf.keras.layers.Input(name='userRatedMovie3', shape=(), dtype='int32'),
    'userRatedMovie4': tf.keras.layers.Input(name='userRatedMovie4', shape=(), dtype='int32'),
    'userRatedMovie5': tf.keras.layers.Input(name='userRatedMovie5', shape=(), dtype='int32'),

    'userGenre1': tf.keras.layers.Input(name='userGenre1', shape=(), dtype='string'),
    'userGenre2': tf.keras.layers.Input(name='userGenre2', shape=(), dtype='string'),
    'userGenre3': tf.keras.layers.Input(name='userGenre3', shape=(), dtype='string'),
    'userGenre4': tf.keras.layers.Input(name='userGenre4', shape=(), dtype='string'),
    'userGenre5': tf.keras.layers.Input(name='userGenre5', shape=(), dtype='string'),
    'movieGenre1': tf.keras.layers.Input(name='movieGenre1', shape=(), dtype='string'),
    'movieGenre2': tf.keras.layers.Input(name='movieGenre2', shape=(), dtype='string'),
    'movieGenre3': tf.keras.layers.Input(name='movieGenre3', shape=(), dtype='string'),
}

# movie id embedding feature
#movie_col = tf.feature_column.categorical_column_with_identity(key='movieId', num_buckets=1001)
#movie_emb_col = tf.feature_column.embedding_column(movie_col, EMBEDDING_SIZE)

# user id embedding feature
user_col = tf.feature_column.categorical_column_with_identity(key='userId', num_buckets=30001)
user_emb_col = tf.feature_column.embedding_column(user_col, EMBEDDING_SIZE)

# genre features vocabulary
genre_vocab = ['Film-Noir', 'Action', 'Adventure', 'Horror', 'Romance', 'War', 'Comedy', 'Western', 'Documentary',
               'Sci-Fi', 'Drama', 'Thriller',
               'Crime', 'Fantasy', 'Animation', 'IMAX', 'Mystery', 'Children', 'Musical']
# user genre embedding feature
user_genre_col = tf.feature_column.categorical_column_with_vocabulary_list(key="userGenre1",
                                                                           vocabulary_list=genre_vocab)
user_genre_emb_col = tf.feature_column.embedding_column(user_genre_col, EMBEDDING_SIZE)
# item genre embedding feature
item_genre_col = tf.feature_column.categorical_column_with_vocabulary_list(key="movieGenre1",
                                                                           vocabulary_list=genre_vocab)
item_genre_emb_col = tf.feature_column.embedding_column(item_genre_col, EMBEDDING_SIZE)


'''
candidate_movie_col = [tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_identity(key='movieId', num_buckets=1001,default_value=0))]
recent_rate_col = [
    tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_identity(key='userRatedMovie1', num_buckets=1001,default_value=0)),
    tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_identity(key='userRatedMovie2', num_buckets=1001,default_value=0)),
    tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_identity(key='userRatedMovie3', num_buckets=1001,default_value=0)),
    tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_identity(key='userRatedMovie4', num_buckets=1001,default_value=0)),
    tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_identity(key='userRatedMovie5', num_buckets=1001,default_value=0)),
]
'''


candidate_movie_col = [ tf.feature_column.numeric_column(key='movieId', default_value=0),   ]

recent_rate_col = [
    tf.feature_column.numeric_column(key='userRatedMovie1', default_value=0),
    tf.feature_column.numeric_column(key='userRatedMovie2', default_value=0),
    tf.feature_column.numeric_column(key='userRatedMovie3', default_value=0),
    tf.feature_column.numeric_column(key='userRatedMovie4', default_value=0),
    tf.feature_column.numeric_column(key='userRatedMovie5', default_value=0),
]



# user profile
user_profile = [
    user_emb_col, #用户id的密集向量 10
    user_genre_emb_col,#用户喜欢题材的密集向量 10
    tf.feature_column.numeric_column('userRatingCount'),
    tf.feature_column.numeric_column('userAvgRating'),
    tf.feature_column.numeric_column('userRatingStddev'),
]

# context features
context_features = [
    item_genre_emb_col,#电影题材的密集向量 是10
    tf.feature_column.numeric_column('releaseYear'),
    tf.feature_column.numeric_column('movieRatingCount'),
    tf.feature_column.numeric_column('movieAvgRating'),
    tf.feature_column.numeric_column('movieRatingStddev'),
]

candidate_layer = tf.keras.layers.DenseFeatures(candidate_movie_col)(inputs)#电影id
user_behaviors_layer = tf.keras.layers.DenseFeatures(recent_rate_col)(inputs) #用户最近5个电影评分
user_profile_layer = tf.keras.layers.DenseFeatures(user_profile)(inputs) #用户特征
context_features_layer = tf.keras.layers.DenseFeatures(context_features)(inputs) #上下文特征

# Activation Unit
#把电影id，和用户对某部电影的评分数量共同作为特征输入
movie_emb_layer = tf.keras.layers.Embedding(input_dim=1001,output_dim=EMBEDDING_SIZE,mask_zero=True)# mask zero

user_behaviors_emb_layer = movie_emb_layer(user_behaviors_layer) 

candidate_emb_layer = movie_emb_layer(candidate_layer) 
candidate_emb_layer = tf.squeeze(candidate_emb_layer,axis=1)  #扩维后变为（None,1,10)
#RECENT_MOVIES为5，就把向量重复5次，得到（None,5,10)
repeated_candidate_emb_layer = tf.keras.layers.RepeatVector(RECENT_MOVIES)(candidate_emb_layer)
#把user_behaviors_emb_layer和user_behaviors_emb_layer相减
activation_sub_layer = tf.keras.layers.Subtract()([user_behaviors_emb_layer,
                                                   user_behaviors_emb_layer])  # element-wise sub
# user_behaviors_emb_layer和repeated_candidate_emb_layer其实可以用一套
activation_product_layer = tf.keras.layers.Multiply()([user_behaviors_emb_layer,
                                                       repeated_candidate_emb_layer])  # element-wise product
#下面每一个都是长度为10的密集向量，合并后变为长度为40的密集向量
activation_all = tf.keras.layers.concatenate([activation_sub_layer, user_behaviors_emb_layer,
                                              repeated_candidate_emb_layer, activation_product_layer], axis=-1)

activation_unit = tf.keras.layers.Dense(32)(activation_all)
activation_unit = tf.keras.layers.PReLU()(activation_unit)  #对每一个输出做一个激活
activation_unit = tf.keras.layers.Dense(1, activation='sigmoid')(activation_unit)
activation_unit = tf.keras.layers.Flatten()(activation_unit)
activation_unit = tf.keras.layers.RepeatVector(EMBEDDING_SIZE)(activation_unit)  #RepeatVector是向量重复，看官网实例
activation_unit = tf.keras.layers.Permute((2, 1))(activation_unit) #根据指定轴，重新排列形状，等价于轴滚动
activation_unit = tf.keras.layers.Multiply()([user_behaviors_emb_layer, activation_unit])
#上面完成了注意力的计算
# sum pooling
user_behaviors_pooled_layers = tf.keras.layers.Lambda(lambda x: tf.keras.backend.sum(x, axis=1))(activation_unit)
#user_behaviors_pooled_layers输出维度为(None, 10) ，某部电影id，用户最近评价的5部电影的
# fc layer，user_profile_layer是23，user_behaviors_pooled_layers是（None,10),candidate_emb_layer是10个，context_features_layer14
#4个分别是 用户属性特征（User Proflie Features）、用户行为特征（User Behaviors）、候选广告特征（Candidate Ad）和场景特征（Context Features）。
concat_layer = tf.keras.layers.concatenate([user_profile_layer, user_behaviors_pooled_layers,
                                            candidate_emb_layer, context_features_layer])
output_layer = tf.keras.layers.Dense(128)(concat_layer)
output_layer = tf.keras.layers.PReLU()(output_layer) #LeakyReLU
output_layer = tf.keras.layers.Dense(64)(output_layer)
output_layer = tf.keras.layers.PReLU()(output_layer)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(output_layer)

model = tf.keras.Model(inputs, output_layer)
print(model.layers)
print(model.summary())
from tensorflow.keras.utils import plot_model

plot_model(model,to_file='din.png')
# compile the model, set loss function, optimizer and evaluation metrics
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', tf.keras.metrics.AUC(curve='ROC'), tf.keras.metrics.AUC(curve='PR')])

# train the model
model.fit(train_dataset, epochs=5)

# evaluate the model
test_loss, test_accuracy, test_roc_auc, test_pr_auc = model.evaluate(test_dataset)
print('\n\nTest Loss {}, Test Accuracy {}, Test ROC AUC {}, Test PR AUC {}'.format(test_loss, test_accuracy,
                                                                                   test_roc_auc, test_pr_auc))

# print some predict results
predictions = model.predict(test_dataset)
for prediction, goodRating in zip(predictions[:12], list(test_dataset)[0][1][:12]):
    print("Predicted good rating: {:.2%}".format(prediction[0]),
          " | Actual rating label: ",
          ("Good Rating" if bool(goodRating) else "Bad Rating"))


[<keras.engine.input_layer.InputLayer object at 0x000002904DDEC160>, <keras.engine.input_layer.InputLayer object at 0x0000029075A00460>, <keras.engine.input_layer.InputLayer object at 0x0000029075A007F0>, <keras.engine.input_layer.InputLayer object at 0x0000029075A00B80>, <keras.engine.input_layer.InputLayer object at 0x00000290759F08E0>, <keras.engine.input_layer.InputLayer object at 0x00000290759D26A0>, <keras.engine.input_layer.InputLayer object at 0x00000290759D23A0>, <keras.engine.input_layer.InputLayer object at 0x00000290759F0550>, <keras.engine.input_layer.InputLayer object at 0x00000290759D2A60>, <keras.engine.input_layer.InputLayer object at 0x00000290759FC250>, <keras.engine.input_layer.InputLayer object at 0x00000290759FC5E0>, <keras.engine.input_layer.InputLayer object at 0x00000290759FC970>, <keras.engine.input_layer.InputLayer object at 0x00000290759FCD00>, <keras.engine.input_layer.InputLayer object at 0x0000029075A000D0>, <keras.engine.input_layer.InputLayer object at 

C:\Users\Administrator\AppData\Roaming\Python\Python39\site-packages\keras\engine\functional.py:559: UserWarning: Input dict contained keys ['rating', 'timestamp', 'userAvgReleaseYear', 'userReleaseYearStddev'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


7403/7403 [==============================] - 79s 10ms/step - loss: 3.9930 - accuracy: 0.5823 - auc: 0.5940 - auc_1: 0.6375

KeyboardInterrupt: 